# Encrypt Client

In [6]:
import requests

## 1. Get public key from server

In [7]:
public_key = requests.get('http://localhost:8080/getPublicKey')
print(public_key)
print()
print(public_key.text)

<Response [200]>

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvj2YjA2ElbtszsUQCjzB\nMz4Yfc2em9D7Kn8Y2lbcUT0KYFEOlylpq8b9785D1nCS08KkzVOJzZlG5ufCSjVA\nbspfTv3+D6sf//gezEeHAH5fnBb7zY5YaERzNOYBcH+RwUFhv5MK/fWXVV/v487u\neQTxyn0uZquOPfEVas9KYgD3yr6/j6idsPnQn7/z7V4QbwTTy9Ox234MX3FTblHX\n276c23J/zGwNN08ZwhffUkxuZoChXC0tqQLwkr2RmRnqE95YE+vyOqs6fot2h1Wy\nCiexirGs4SyfxVHCiAzxsasFoMtsBP2ANkMiTiWb8Us3+D96Zx84NWVa0h3cIsmf\nuQIDAQAB\n-----END PUBLIC KEY-----\n'


## 2. Generate a symmetric key and use the server public key to encrypt it

In [8]:
import os
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key

def encrypt_symmetric_key_with_public_key(public_key_pem, symmetric_key):
    public_key = load_pem_public_key(public_key_pem)

    encrypted_key = public_key.encrypt(
        symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_key

# 客户端生成一个随机的AES密钥
aes_key = os.urandom(32)  # 例如，使用32字节的密钥

print(('\n' + public_key.text[2:-1].replace("\\n", "\n")).encode())

# 加密AES密钥
encrypted_aes_key = encrypt_symmetric_key_with_public_key(('\n' + public_key.text[2:-1].replace("\\n", "\n")).encode(), aes_key)

print()
print(aes_key)
print()
print(encrypted_aes_key)

b'\n-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvj2YjA2ElbtszsUQCjzB\nMz4Yfc2em9D7Kn8Y2lbcUT0KYFEOlylpq8b9785D1nCS08KkzVOJzZlG5ufCSjVA\nbspfTv3+D6sf//gezEeHAH5fnBb7zY5YaERzNOYBcH+RwUFhv5MK/fWXVV/v487u\neQTxyn0uZquOPfEVas9KYgD3yr6/j6idsPnQn7/z7V4QbwTTy9Ox234MX3FTblHX\n276c23J/zGwNN08ZwhffUkxuZoChXC0tqQLwkr2RmRnqE95YE+vyOqs6fot2h1Wy\nCiexirGs4SyfxVHCiAzxsasFoMtsBP2ANkMiTiWb8Us3+D96Zx84NWVa0h3cIsmf\nuQIDAQAB\n-----END PUBLIC KEY-----\n'

b'vr\xed\xeeJ\x10\x98\xbb\x1d\xe1R2/]DG\xfd\x04fr\x81\x8e\xb5\x9e\x0b\xda\xb8\x1b\xd7rK\xf6'

b'\x03[\xc4\x8aX\xd2>\xb6\x1e&\x858\xb2o\n\xdf\xeb\x88\xda\xbd\xe0\xbdP\x9c\xfd\xb4\x97\xe1\x84^\xcdw/\xed\'hR\xfe/\xf3\xbaq\xfe\xa3\x04\x1f\x01{u5\xa7\xd8\xaa\t\x8e\xe3\x05\xc1\xa5\xa2\xaf\xb9VjV\x8c8)\xe3\xb53\x14d\xe9\x9ba\x97\x0el\x9a7B\x983\xdd=:@K\xe0\xcc\xed\x94Qi\xfa\xaf\xd8\\\xed\xa23p\x05\x01v\xca\xb2\x82\x06N\xc0\xfcs*]\x9795\x00\xbf\xe7;\x93\xa7\xfe\xa0\xdf-\x07_\xee\xbf\xf8A\xca\x0e4\xf7\x93\x148^\x87:\x01\x93\x81\x19\xbe#

## 3. Send encrypted key to server

In [9]:
res = requests.post('http://localhost:8080/sendEncryptKey', data=encrypted_aes_key)
print(res)
print(res.text)

<Response [200]>
Send Success!


## 4. Encrypted communication with the server

In [10]:
import AES
import io

with open("tmp/a.txt", 'rb') as file:
    file_data = file.read()
encrypted_data = AES.aes_encrypt(aes_key, file_data)
# 创建一个内存中的字节流
encrypted_file = io.BytesIO(encrypted_data)
files = {'file': ('a.txt', encrypted_file)} # 使用内存中的字节流

headers = {"Authorization": "Basic Y2xpZW50MToxMjM=", "X-Content-Encrypted": "true"}
res = requests.post(url='http://127.0.0.1:8080/upload?path=client1/', headers=headers, files=files)
print(res)
print(AES.aes_decrypt(aes_key, res.content))

headers = {"Authorization": "Basic Y2xpZW50MToxMjM=", "X-Content-Encrypted": "true"}
res = requests.get(url='http://127.0.0.1:8080/client1/a.txt', headers=headers)
print(AES.aes_decrypt(aes_key, res.content))

<Response [200]>
b'Upload success!'
b'sadfsdfasdf'
